# Create Meta Data for Cryptol training examples

## McCabe Cyclomatic Complexity
The json files being used to compute the MCC are in the following format:

```json
{
    "definitions": [
        {
            "kind": "",
            "locals": [],
            "mcc": [] | null,
            "name": "",
            "params": [],
            "references": [],
            "signature": null
        }
    ],
    "imports": [],
    "filename": ""
}
```
This notebook creates the json above for each training example, creates a dataframe to store the json path, filename, and MCC summary statistics for each training example.

In [1]:
from pathlib import Path 
import os, dotenv, yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

dotenv.load_dotenv()
os.chdir(Path(config["pythonpath"]).expanduser())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [3]:
import subprocess
import pandas as pd

VERSION = config["version"]
REPO_ROOT_DIR = Path(config["repo_root"]).expanduser()

MCC_ROOT = config["mcc_root"]
MCC_ROOT_DIR = Path(MCC_ROOT).expanduser()
MCC_ROOT_DIR.mkdir(parents=True, exist_ok=True)

clean_data_df = pd.read_json(
    "data/clean_datasets/verified_nocomments_v1-0.jsonl", 
    lines=True
    )

CABAL_PROJECT_ROOT = "toy-cryptol-ast" 

clean_data_df.head()

,filename,filetype,content,variant
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([...",without_comments
1,cryptol/examples/AE.cry,cry,module AE where\n\nparameter\n type A : * ...,without_comments
2,cryptol/examples/Cipher.cry,cry,module Cipher where\n\ntype Cipher KeySize Blo...,without_comments
3,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...,without_comments
4,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...,without_comments


In [4]:
def run_haskell_mcc_generator(cry_path: Path):
    """
    Call: cabal run toy-cryptol-ast -- <file> <OUTDIR>
    from within CABAL_PROJECT_ROOT.
    """

    cmd = [
        "cabal",
        "run",
        "cryptol-meta",
        "--",
        str(cry_path),
    ]

    print("Running:", " ".join(cmd))
    result = subprocess.run(
        cmd,
        cwd=str(CABAL_PROJECT_ROOT),  # run inside the cabal project
        capture_output=True,
        text=True,
    )

    if result.returncode != 0:
        print(f"❌ Error ({result.returncode}) on {cry_path}")
        if result.stdout.strip():
            print("STDOUT:\n", result.stdout)
        if result.stderr.strip():
            print("STDERR:\n", result.stderr)
    else:
        print(f"✅ OK: {cry_path}")
        if result.stdout.strip():
            print("STDOUT:\n", result.stdout)
    return result


In [9]:

clean_data_df = clean_data_df.iloc[ : 10]
for index, row in clean_data_df.iterrows():
    cry_path = Path(row["filename"])
    output_file = MCC_ROOT_DIR / f"{cry_path.stem}.json"
    output_file.parent.mkdir(parents=True, exist_ok=True)
    res = run_haskell_mcc_generator(REPO_ROOT_DIR / cry_path)
    mcc = json.loads(res.stdout)
    output_file.write_text(
        json.dumps(mcc, indent=2),
        encoding="utf-8"
        )

Running: cabal run cryptol-meta -- /Users/josh/Automated_Reasoning_for_Cryptography/cryptol/examples/splitAt.cry
✅ OK: /Users/josh/Automated_Reasoning_for_Cryptography/cryptol/examples/splitAt.cry
STDOUT:
 {"definitions":[{"kind":"declaration","locals":[],"mcc":{"edges":[{"from":1,"kind":"control","label":null,"to":2},{"from":2,"kind":"control","label":null,"to":3},{"from":3,"kind":"control","label":null,"to":4},{"from":4,"kind":"control","label":null,"to":5},{"from":0,"kind":"control","label":null,"to":1},{"from":5,"kind":"control","label":null,"to":6}],"entry":0,"exit":6,"nodes":[{"details":[],"id":0,"kind":"entry","label":"x"},{"details":[],"id":1,"kind":"literal","label":"lit"},{"details":[],"id":2,"kind":"literal","label":"lit"},{"details":[],"id":3,"kind":"literal","label":"lit"},{"details":[],"id":4,"kind":"literal","label":"lit"},{"details":["arity=4"],"id":5,"kind":"list","label":"list"},{"details":[],"id":6,"kind":"exit","label":"exit"}]},"name":"x","params":[],"references":[